<a href="https://colab.research.google.com/github/jakelaporte/MathematicalModeling/blob/master/Lsn10_SensitivityAnalysis_ShadowPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sensitivity Analysis and Shadow Prices
The purpose of this lesson is to figure out what happens to the objective function when our assumptions about the right-hand side (rhs) of the constraint equations are perturbed by a small amount. We start by reproducing (correctly) what they are trying to do in the book (the book left off the constraints $x_1\le5000$ and $x_2\le8000$). I assume because they were not binding at the optimum, but as the elasticity changes, they become binding - see below.<br><br>
In most applications, only approximate data is available. Measurement errors, fluctuations in data, unavailability of information are some of the factors that contribute to imprecision in the optimization model. In the absence of precise data, there may be no choice but to solve the model with the best available estimates. Once the solution is obtained, it is reasonable to assess the quality of the resulting solution. A key question is, how sensitive is the solution to variations in the data? In this notebook, we first look at how changes in the assumptions that make up the objective function could change the solution, then we turn to looking at the changes in the rhs of the constraints and figure out why these are called shadow prices.

In [0]:
## a = 0.01 ##
options(warn=-1)
library(MASS);library(NlcOptim);library(ma391laporte)
## Example 2.2 - Meerschaert ##
P = function(x){
    return(((339-0.01*x[1]-0.003*x[2])*x[1]
           +(399-0.004*x[1]-0.01*x[2])*x[2]
           -(400000+195*x[1]+225*x[2]))*-1)
}
###Linear Inequality Constraints##
x0=c(1000,1000)
A = matrix(c(1,0,1,-1,0,0,1,1,0,-1),nrow=5) #defining this matrix is not intuitive - make sure we talk in class
B = matrix(c(5000,8000,10000,0,0),nrow=5)
print(A)
print(B)
solnl(x0,objfun=P,A=A,B=B)

     [,1] [,2]
[1,]    1    0
[2,]    0    1
[3,]    1    1
[4,]   -1    0
[5,]    0   -1
      [,1]
[1,]  5000
[2,]  8000
[3,] 10000
[4,]     0
[5,]     0


$par
         [,1]
[1,] 3846.154
[2,] 6153.846

$fn
[1] -532307.7

$counts
     nfval ngval
[1,]    19     7

$lambda
$lambda$lower
     [,1]
[1,]    0
[2,]    0

$lambda$upper
     [,1]
[1,]    0
[2,]    0

$lambda$ineqlin
[1]  0  0 24  0  0


$grad
          [,1]
[1,] -24.00000
[2,] -23.99999

$hessian
            [,1]        [,2]
[1,] 0.020029796 0.007039313
[2,] 0.007039313 0.020238593

In [0]:
## Change a and see what it does to the objective function ###
f = function(a){
    P = function(x){
    return(((339-a*x[1]-0.003*x[2])*x[1]
           +(399-0.004*x[1]-0.01*x[2])*x[2]
           -(400000+195*x[1]+225*x[2]))*-1)
    }
    x0=c(1000,1000)
    A = matrix(c(1,0,1,-1,0,0,1,1,0,-1),nrow=5)
    B = matrix(c(5000,8000,10000,0,0),nrow=5)
    ans = solnl(x0,objfun=P,A=A,B=B)
    return(ans)
}
ans.x1=0
ans.x2=0
ans.profit=0
sa = seq(0.001,0.025,0.001)
for (i in 1:length(sa)){
    res = f(sa[i])
    ans.x1[i]=res$par[1]
    ans.x2[i]=res$par[2]
    ans.profit[i]= -res$fn
}
result = data.frame(a = sa,x1=ans.x1,x2=ans.x2,profit = ans.profit)
print(result)

       a       x1       x2   profit
1  0.001 5000.000 5000.000 740000.0
2  0.002 5000.000 5000.000 715000.0
3  0.003 5000.000 5000.000 690000.0
4  0.004 5000.000 5000.000 665000.0
5  0.005 5000.000 5000.000 640000.0
6  0.006 5000.000 5000.000 615000.0
7  0.007 5000.000 5000.000 590000.0
8  0.008 4545.455 5454.545 567272.7
9  0.009 4166.667 5833.333 548333.3
10 0.010 3846.154 6153.846 532307.7
11 0.011 3571.428 6428.572 518571.4
12 0.012 3333.333 6666.667 506666.7
13 0.013 3125.000 6875.000 496250.0
14 0.014 2941.176 7058.824 487058.8
15 0.015 2777.778 7222.222 478888.9
16 0.016 2631.579 7368.421 471578.9
17 0.017 2500.000 7500.000 465000.0
18 0.018 2380.952 7619.048 459047.6
19 0.019 2272.727 7727.273 453636.4
20 0.020 2173.913 7826.087 448695.7
21 0.021 2083.333 7916.667 444166.7
22 0.022 2000.000 8000.000 440000.0
23 0.023 1913.043 8000.000 436173.9
24 0.024 1833.333 8000.000 432666.7
25 0.025 1760.000 8000.000 429440.0


### Binding / Active Constraints
The following lists the binding constraints for the problem at different elasticity levels.

|a |# binding constraint           |
|--|-------------------------------------------|
|0.001|$x_1=5000$, and  $x_1+x_2 = 10000$   |
|...|...|
|0.008|$x_1+x_2=10000$|
|...|...|
|0.022|$x_2=8000$,   $x_1+x_2=10000$     |
|0.023|$x_2=8000$|
|...|...|
|0.025|$x_2=8000$|



## Shadow Price - Review MA205/255 (Constrained Optimization with multiple constraints)
A Lagrange multiplier problem with the following setup:
$$\min_{\vec{x}}{f(\vec{x})}$$
$$\text{s.t. }g_1(\vec{x})=b_1 $$
$$g_2(\vec{x})=b_2$$
$$...$$
$$g_m(\vec{x})=b_m$$<br>
is solved using Lagrange multipliers like we covered in class by finding the solution to the following equations:
$$\vec{\triangledown}f(\vec{x})=\lambda_1\vec{\triangledown}g_1(\vec{x})+\lambda_2\vec{\triangledown}g_2(\vec{x})+...+ \lambda_m\vec{\triangledown}g_m(\vec{x})$$
$$g_1(\vec{x})=b_1$$
$$g_2(\vec{x})=b_2$$
$$...$$

If we let all of our constraint functions ($g_1(\vec{x}),g_2(\vec{x}),...$) be linear, then we can turn this list of constraints into matrix notation.



$$\min_{\vec{x}}{f(\vec{x})}$$
$$ \text{s.t. }A\vec{x}=\vec{b} $$
- Assume $f$ is twice differentiable, and $A$ is an ```mxn``` matrix with full row rank
- Assume a local minimizer is found $\vec{x}_*$ that minimizes the function and satisfies the constraint $A\vec{x}_*=\vec{b}$.<br><br>
Let's look at an example, to see what this looks like for a set of linear constraint equations.

#### Simple example ####
$$\min_{\vec{x}}{f(\vec{x})}$$<br>
$$ \text{s.t. }3x_1+2x_2-x_3=5$$
$$ -2x_1+x_2+2x_3=12$$<br>
Turning these constraints into matrix form, $A\vec{x}=\vec{b}$, we get:
$$\begin{bmatrix}3&2&-1\\-2&1&2\end{bmatrix} \begin{bmatrix}x_1\\x_2\\x_3\end{bmatrix}=\begin{bmatrix}5\\12\end{bmatrix}$$<br><br>
so that $\vec{\triangledown}g_1(\vec{x})=\begin{bmatrix}3\\2\\-1\end{bmatrix} \text{ and }\vec{\triangledown}g_2(\vec{x})=\begin{bmatrix}-2\\1\\-2\end{bmatrix}$.<br><br>
If we are solving this with Lagrange multipliers, we get the following set of equations and $\vec{x}_*$ represents the solution to the equations which is the optimum:
$$\vec{\triangledown}f(\vec{x}_*)=\lambda_{1*}\begin{bmatrix}3\\2\\-1\end{bmatrix} + \lambda_{2*}\begin{bmatrix}-2\\1\\-2\end{bmatrix} = \begin{bmatrix}3&-2\\2&1\\-1&2\end{bmatrix}\begin{bmatrix}\lambda_{1*} \\ \lambda_{2*}\end{bmatrix}=A^T \vec{\lambda}_*$$<br><br>
Note the following from using Lagrange multipliers on a linear set of constraints:
$$\boxed{\vec{\triangledown}f(\vec{x}_*)=A^T \vec{\lambda}_*}$$

## Perturbations in RHS of constraints
Now suppose we introduce a small perturbation(change) in the $\vec{b}$ of the constraints, the question is - how does this small change effect our solution? This is the same question that we ask in our sensitivity analysis. Luckily, there is an easy answer to this question, but we will need a little math in order to understand why it is true. Let's start by perturbing $\vec{b}$ by a small amount $\vec{b}+\vec{\delta}$ where $\vec{\delta}$ is a vector of small values. If these are sufficiently small, then we assume that the new optimum ($\bar{x}$) is close to $\vec{x}_*$. For $\bar{x}$ close to $\vec{x}_*$ with $A\bar{x}=\vec{b}+\vec{\delta}$, use the Taylor Series approximation to obtain:<br><br>
$$f(\vec{x}_*+(\bar{x}-\vec{x}_*))= f(\bar{x})\approx f(\vec{x}_*)+(\bar{x}-\vec{x}_*)^T\vec{\triangledown}f(\vec{x}_*)$$<br>
Using our boxed equation from above, we see that:
$$ = f(\vec{x}_*)+(\bar{x}-\vec{x}_*)^TA^T \vec{\lambda}_*$$<br><br>
Taking a closer look at the transpose of $(\bar{x}-\vec{x}_*)^TA^T$, we see the following:
$$A(\bar{x}-\vec{x}_*) =  \underbrace{ A\bar{x}}_{\vec{b}+\vec{\delta}}   -   \underbrace{A\vec{x}_*}_{\vec{b}}   = \vec{b}+\vec{\delta}-\vec{b}=\vec{\delta}$$<br><br>
This shows that $(\bar{x}-\vec{x}_*)^TA^T = \vec{\delta}^T$<br><br>
$$f(\bar{x})\approx f(\vec{x}_*)+   \underbrace{(\bar{x}-\vec{x}_*)^TA^T}_{\vec{\delta}^T}    \vec{\lambda}_*=f(\vec{x}_*)+  \underbrace{\vec{\delta}^T\vec{\lambda}_*}_{\text{dot product}}
$$ <br><br>
This shows that small perturbations in the rhs of the constraint equation, leads to changes that are proportional to the Lagrange multiplier at the optimum ($\lambda_*$). Let's make changes to our TV example and note how these changes effect our solution in terms of the shadow prices.

## Shadow Price Example
From page 47, the **shadow price** represents the value to the company of a certain resource (constraint). If the company is interested in increasing its profit, then it should investigate the binding constraints on the optimal values.<br><br>
$$\max_{x_1,x_2}{(339-0.01x_1-0.003x_2)x_1+(339-0.004x_1-0.01x_2)x_2-(400000+195x_1+225x_2)} $$
$$ \text{s.t. }x_1+x_2\le10,000 \text{ (production capacity)}$$ 
$$ x_1\le5,000 \text{ (max 19in TVs)}$$
$$ x_2\le8,000\text{ (max 21in TVs)}$$<br>
The only binding constraint for the solution is the production capacity of $x_1+x_2\le10,000$ which had a $\lambda$ value of 24. The meaning of this that increasing production capacity by 1 would increase the profit by ```$24```. Let's change the constraint on the problem and let ```R``` show us what happens to the profit. Let $x_1+x_2=10,001$ and solve for the new profit.

In [0]:
## Perturbed problem to denote what happens when production capacity changes.
P = function(x){
    return(((339-0.01*x[1]-0.003*x[2])*x[1]
           +(399-0.004*x[1]-0.01*x[2])*x[2]
           -(400000+195*x[1]+225*x[2]))*-1)
}
###Linear Inequality Constraints##
x0=c(1000,1000)
A = matrix(c(1,0,1,-1,0,0,1,1,0,-1),nrow=5) #defining this matrix is not intuitive - make sure we talk in class
B = matrix(c(5000,8000,10001,0,0),nrow=5)
print(A)
print(B)
solnl(x0,objfun=P,A=A,B=B)

     [,1] [,2]
[1,]    1    0
[2,]    0    1
[3,]    1    1
[4,]   -1    0
[5,]    0   -1
      [,1]
[1,]  5000
[2,]  8000
[3,] 10001
[4,]     0
[5,]     0


$par
         [,1]
[1,] 3846.654
[2,] 6154.346

$fn
[1] -532331.7

$counts
     nfval ngval
[1,]    19     7

$lambda
$lambda$lower
     [,1]
[1,]    0
[2,]    0

$lambda$upper
     [,1]
[1,]    0
[2,]    0

$lambda$ineqlin
[1]  0.0000  0.0000 23.9865  0.0000  0.0000


$grad
          [,1]
[1,] -23.98649
[2,] -23.98650

$hessian
            [,1]        [,2]
[1,] 0.020140590 0.007049412
[2,] 0.007049412 0.020105975

#### Results 
|Production Capacity|Profit|$x_1$|$x_2$|
|-------------------|------|-----|-----|
|10,000|```$532,307```|3846|6154|
|10,001|```$532,331```|3846|6154|

532,331-532,307=24<br>
The difference in the profit is ```$24``` which was expected because of the shadow price.

## Example 2.5 - change the constraint on 19 in TVs
$$\max_{x_1,x_2}{(339-0.01x_1-0.003x_2)x_1+(339-0.004x_1-0.01x_2)x_2-(400000+195x_1+225x_2)} $$
$$ \text{s.t. }x_1+x_2\le10,000 \text{ (production capacity)}$$ 
$$ x_1\le3,000 \text{ (max 19in TVs)}$$
$$ x_2\le8,000\text{ (max 21in TVs)}$$<br>

In [0]:
P = function(x){
    return(((339-0.01*x[1]-0.003*x[2])*x[1]
           +(399-0.004*x[1]-0.01*x[2])*x[2]
           -(400000+195*x[1]+225*x[2]))*-1)
}
###Linear Inequality Constraints##
x0=c(1000,1000)
A = matrix(c(1,0,1,-1,0,0,1,1,0,-1),nrow=5) #defining this matrix is not intuitive - make sure we talk in class
B = matrix(c(3000,8000,10000,0,0),nrow=5)
solnl(x0,objfun=P,A=A,B=B)

$par
     [,1]
[1,] 3000
[2,] 7000

$fn
[1] -523000

$counts
     nfval ngval
[1,]    16     6

$lambda
$lambda$lower
     [,1]
[1,]    0
[2,]    0

$lambda$upper
     [,1]
[1,]    0
[2,]    0

$lambda$ineqlin
[1] 22  0 13  0  0


$grad
     [,1]
[1,]  -35
[2,]  -13

$hessian
            [,1]        [,2]
[1,] 0.075464446 0.007000001
[2,] 0.007000001 0.019999998

### Results - two constraints are binding
Now the constraints $x_1+x_2\le10,000$ and $x_1\le3,000$ are binding and there is a shadow price for each of these.

***
# ==================================================
# ========  Homework Lesson 10  ========================
# ==================================================
***

#### Problem 1: Using this new constraint from example 2.5, use R to show how changing the rhs of the constraints changes the objective function.
$$\max_{x_1,x_2}{(339-0.01x_1-0.003x_2)x_1+(339-0.004x_1-0.01x_2)x_2-(400000+195x_1+225x_2)} $$
$$ \text{s.t. }x_1+x_2\le10,001 \text{ (production capacity)}$$ 
$$ x_1\le3,001 \text{ (max 19in TVs)}$$
$$ x_2\le8,001\text{ (max 21in TVs)}$$<br>
#### Problem2: Explain why the solution that you got in R either confirmed what you thought was going to happen and why; or did not confirm and possible reasons why.

#### Problem3: Go back to the originial problem and create a table of rhs for the $x_1+x_2=10000$ constraint - keep track of $x_1,x_2,\lambda_{\text{constraint}},\text{profit}$. Make a reasonable window of possible rhs's for this constraint and explain what happens. Does shadow price stay at 24? If not, explain why.

#### Problem 4: Create your own optimization problem with a constraint. Find its shadow prices (implies that you need at least one constraint to be binding) and verify that an increase in the constraint (by 1) actually increases the objective function by the shadow price.